In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import sagemaker

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
df = pd.read_csv('housing.csv')
df.head()

,crim,zn,indus,chas,nox,age,rm,dis,rad,tax,ptratio,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,5.33,36.2


In [4]:
dataset = pd.concat([df['medv'], df.drop(['medv'], axis=1)], axis=1)

In [6]:
dataset.head()

,medv,crim,zn,indus,chas,nox,age,rm,dis,rad,tax,ptratio,lstat
0,24.0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,4.98
1,21.6,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,9.14
2,34.7,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,4.03
3,33.4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,2.94
4,36.2,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,5.33


In [7]:
training_dataset, validation_dataset = train_test_split(dataset, test_size=0.1)
print(training_dataset.shape)
print(validation_dataset.shape)

(455, 13)
(51, 13)


In [8]:
# Save training and validation datasets as CSV files
training_dataset.to_csv('training_dataset.csv', index=False, header=False)
validation_dataset.to_csv('validation_dataset.csv', index=False, header=False)

In [9]:
sess = sagemaker.Session()
bucket = sess.default_bucket()

prefix = 'boston-housing'

training_data_path = sess.upload_data(path='training_dataset.csv', key_prefix=prefix + '/input/training')
validation_data_path = sess.upload_data(path='validation_dataset.csv', key_prefix=prefix + '/input/validation')
print(training_data_path)
print(validation_data_path)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
s3://sagemaker-us-east-1-466175814801/boston-housing/input/training/training_dataset.csv
s3://sagemaker-us-east-1-466175814801/boston-housing/input/validation/validation_dataset.csv


In [12]:
import boto3
from sagemaker.estimator import Estimator

region = boto3.Session().region_name    
print(region)
container = sagemaker.image_uris.retrieve("xgboost", region, "1.7-1")
print(container)

xgb_estimator = Estimator(container,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=f's3://{bucket}/{prefix}/output'
)

xgb_estimator.set_hyperparameters(
    objective='reg:squarederror',
    num_round=100,                  # number of boosting rounds
    max_depth=5                     # maximum tree depth
)

us-east-1
683313688378.dkr.ecr.us-east-1.amazonaws.com/sagemaker-xgboost:1.7-1
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [14]:
# Create TrainingInput channels
from sagemaker.inputs import TrainingInput

training_data_channel = TrainingInput(s3_data=training_data_path, content_type='csv')
validation_data_channel = TrainingInput(s3_data=validation_data_path, content_type='csv')

In [15]:
# Fit the XGBoost model
xgb_estimator.fit({'train': training_data_channel, 'validation': validation_data_channel})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2023-12-18-18-32-20-292


2023-12-18 18:32:20 Starting - Starting the training job...
2023-12-18 18:32:37 Starting - Preparing the instances for training.........
2023-12-18 18:34:05 Downloading - Downloading input data...
2023-12-18 18:34:50 Downloading - Downloading the training image........
2023-12-18 18:36:22 Training - Training image download completed. Training in progress.
2023-12-18 18:36:22 Uploading - Uploading generated training model
2023-12-18 18:36:22 Completed - Training job completed
[2023-12-18 18:36:01.044 ip-10-0-172-206.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-12-18 18:36:01.067 ip-10-0-172-206.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2023-12-18:18:36:01:INFO] Imported framework sagemaker_xgboost_container.training
[2023-12-18:18:36:01:INFO] Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
[2023-12-18:18:36:01:INFO] No GPUs detected (normal if no gpus installed)
[2023-

In [16]:
from time import strftime, gmtime
timestamp = strftime('%d-%H-%M-%S', gmtime())

endpoint_name = 'xgb-demo-'+timestamp
print(endpoint_name)

xgb-demo-18-18-36-33


In [17]:
xgb_predictor = xgb_estimator.deploy(endpoint_name=endpoint_name, 
                        initial_instance_count=1, 
                        instance_type='ml.m5.large')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2023-12-18-18-36-33-232
INFO:sagemaker:Creating endpoint-config with name xgb-demo-18-18-36-33
INFO:sagemaker:Creating endpoint with name xgb-demo-18-18-36-33


------!

In [18]:
test_sample = '0.00123,12.00,1.3410,0,0.6310,7.4410,60.35,5.0411,1,296.1,05.30,3.19'

xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()
xgb_predictor.deserializer = sagemaker.deserializers.CSVDeserializer()

response = xgb_predictor.predict(test_sample)
print(response)

[['42.542137145996094']]


In [19]:
xgb_predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: xgb-demo-18-18-36-33
INFO:sagemaker:Deleting endpoint with name: xgb-demo-18-18-36-33
